In [4]:
import tweepy
consumer_key="VpjjZzsZCOdcgfAaSsjVovT63"
consumer_secret="PpQ3MoZCQhQV2Kfl9HOHLI3AQVimDAGrd93x2xnrNE7wpCefv6"
access_token="771594014-K2c9gI8hDhmweq4l5c2geg3MloxnIFW3ENa005wg"
access_token_secret="HCUguCfNZaPvt3ibW1pwIxS0Zr7ytRM1jddD3vo8vKais"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
# api.user_timeline()

In [5]:
twitter_handle = 'danskdf1995'
counter = 0
for status in tweepy.Cursor(api.user_timeline, screen_name=twitter_handle, tweet_mode="extended").items():
    print(counter)
    counter = counter + 1
    print(status._json)
    if counter > 10:
        break

0
{'created_at': 'Sat Jun 29 10:22:33 +0000 2019', 'id': 1144914041284026368, 'id_str': '1144914041284026368', 'full_text': 'RT @skaarup_df: Rækken af løftebrud hober sig op i rød blok. Allerede dagen efter regeringsdannelsen trækker SF i land på de før valget så…', 'truncated': False, 'display_text_range': [0, 139], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'skaarup_df', 'name': 'Peter Skaarup', 'id': 3144074691, 'id_str': '3144074691', 'indices': [3, 14]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1227448789, 'id_str': '1227448789', 'name': 'Dansk Folkeparti', 'screen_name': 'DanskDf1995', 'location': '', 'description': 'Denne Twitterprofil administreres af Dansk Folkepartis pressetjeneste.', 'url': 'http://t

In [ ]:

import os
os._exit(00)


In [1]:
from nlp4all.models import TweetTagCategory, Tweet, User, Organization, Project, BayesianAnalysis
from nlp4all import db, bcrypt
import json
from datetime import datetime
import time
import json



/Users/au625741/Documents/git/nlp4all/nlp4all/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [26]:
[t.name for t  in TweetTagCategory.query.all()]

['danskdf1995',
 'sfpolitik',
 'enhedslisten',
 'konservativedk',
 'Spolitik',
 'nyeborgerlige',
 'liberalalliance',
 'radikale',
 'venstredk',
 'alternativet']

In [7]:

org = Organization.query.first()
org.categories = TweetTagCategory.query.all()[:3]

In [21]:
project = Project.query.first()
project.categories = TweetTagCategory.query.all()[:3]
# user = User(name="ahjorth", password=bcrypt.generate_password_hash("Hermes_2014"), email="arthur@cs.au.dk")
# db.session.add(user)

In [23]:
db.session.add(project)
db.session.commit()

# db.session.add(org)
# db.session.commit()
# user = User(username="ahjorth", organization=1, password=bcrypt.generate_password_hash("Hermes_2014"), email="arthur@cs.au.dk")
# db.session.add(user)
# db.session.commit()

# user = User(username="ahjorth2", organization=1, password=bcrypt.generate_password_hash("Hermes_2014"), email="arthur.hjorth@u.northwestern.edu")
# db.session.add(user)
# db.session.commit()


In [24]:
project2 = Project.query.first()
project2.categories

[<TweetTagCategory 1>, <TweetTagCategory 2>, <TweetTagCategory 3>]

In [12]:
TweetTagCategory.query.first().categories

AttributeError: 'TweetTagCategory' object has no attribute 'categories'

In [4]:
Tweet.query.all()

[]

In [5]:
# d ={'a' : 3, 'b' : 2}
# json.dumps(d)
# bm = BayesianAnalysis(data=json.dumps(d))
# db.session.add(bm)
# db.session.commit()
# for b in BayesianModel.query.all():
#     print(b.data)


In [6]:
# this script will import tweets. it takes a list of dictionaries, and puts them
# in the database with the following mappings:
## TweetTagDescription
## name = input.twitter_handle
## text = input.full_text

## Tweet
## content = input.full_text
## twitter_handle = input.handle
## url = input.id  (look below for processing)
## date_posted = input.created_at (look below for processing)
## true_tag = TweetTagDescription(input.twitter_handle)
from nlp4all.models import TweetTagCategory, Tweet, User, Organization, Project, BayesianAnalysis
from nlp4all import db, bcrypt
import json
from datetime import datetime
import time
import json


import os

db.drop_all()
db.create_all()


data_dir = 'tweet_data/'
files = [f for f in os.listdir(data_dir) if '_out.json' in f]

existing_tag_names = []
for f in files:
    with open(data_dir+f) as inf:
        counter = 0
        for line in inf.readlines()[:25]:
            indict = json.loads(line)
#             add cateogyr if it does not already exist
            if indict['twitter_handle'] not in existing_tag_names:
                category = TweetTagCategory.query.filter_by(name = indict['twitter_handle']).first()
                if not category:
                    category = TweetTagCategory(name = indict['twitter_handle'], description = "Tweet from " + indict['twitter_handle'])
                    db.session.add(category)
                    db.session.commit()
                existing_tag_names.append(indict['twitter_handle'])
            category = TweetTagCategory.query.filter_by(name = indict['twitter_handle']).first()
            date_str = indict['time']
            date_rep = '%a %b %d %H:%M:%S %z %Y'
            unix_time = time.mktime(datetime.strptime(date_str, date_rep).timetuple())
            timestamp = datetime.fromtimestamp(unix_time)
            a_tweet = Tweet(
                time_posted = timestamp,
                category = category.id,
                handle = indict['twitter_handle'],
                text= indict['full_text'],
                words = [w for w in indict['full_text'].lower().split() if "#" not in w and "http" not in w],
                links = [w for w in indict['full_text'].split() if "http" in w],
                hashtags = [w for w in indict['full_text'].split() if "#" in w],
                url = "https://twitter.com/"+indict['twitter_handle']+"/"+str(indict['id'])
                )
            
            db.session.add(a_tweet)


db.session.commit()
db.session.close()

In [7]:
Tweet.query.first().words

['dejligt',
 'med',
 'ros',
 'fra',
 'min',
 'borgmester',
 'for',
 'at',
 'have',
 'lyttet',
 'og',
 'ændret',
 'i',
 'sundhedsreformen']

In [6]:
t

In [ ]:

import os
os._exit(00)


In [1]:
from nlp4all import db, bcrypt
from nlp4all.models import User, Role, Organization, Project
from nlp4all.models import TweetTagCategory, Tweet, User, Organization, Project, BayesianAnalysis
from nlp4all import db, bcrypt
import json, os
from datetime import datetime
import time
import json



db.drop_all()
db.create_all()
db.session.commit()

admin_role = Role(name='Admin')
db.session.add(admin_role)
db.session.commit()


student_role = Role(name='Student')
db.session.add(student_role)
db.session.commit()


teacher_role = Role(name='Teacher')
db.session.add(teacher_role)
db.session.commit()



hp = bcrypt.generate_password_hash("Hermes_2014")
user = User(username="arthurhjorth", email="arthur.hjorth@stx.oxon.org", password=hp)
user.roles = [admin_role,]
db.session.add(user)

org = Organization(name="UBI/CCTD")
db.session.add(org)
db.session.commit()

user = User(username="arthurhjorth_teacher", email="arthur.hjorth@u.northwestern.edu", password=hp, organizations=[org,])
user.roles = [teacher_role,]
db.session.add(user)
user = User(username="arthurhjorth_student", email="hermeshjorth2011@u.northwestern.edu", password=hp, organizations=[org,])
user.roles = [student_role,]
db.session.add(user)
db.session.commit()



data_dir = 'tweet_data/'
files = [f for f in os.listdir(data_dir) if '_out.json' in f]

existing_tag_names = []
for f in files:
    with open(data_dir+f) as inf:
        counter = 0
        for line in inf.readlines()[:25]:
            indict = json.loads(line)
#             add cateogyr if it does not already exist
            if indict['twitter_handle'] not in existing_tag_names:
                category = TweetTagCategory.query.filter_by(name = indict['twitter_handle']).first()
                if not category:
                    category = TweetTagCategory(name = indict['twitter_handle'], description = "Tweet from " + indict['twitter_handle'])
                    db.session.add(category)
                    db.session.commit()
                existing_tag_names.append(indict['twitter_handle'])
            category = TweetTagCategory.query.filter_by(name = indict['twitter_handle']).first()
            date_str = indict['time']
            date_rep = '%a %b %d %H:%M:%S %z %Y'
            unix_time = time.mktime(datetime.strptime(date_str, date_rep).timetuple())
            timestamp = datetime.fromtimestamp(unix_time)
            a_tweet = Tweet(
                time_posted = timestamp,
                category = category.id,
                handle = indict['twitter_handle'],
                text= indict['full_text'],
                words = [w for w in indict['full_text'].lower().split() if "#" not in w and "http" not in w],
                links = [w for w in indict['full_text'].split() if "http" in w],
                hashtags = [w for w in indict['full_text'].split() if "#" in w],
                url = "https://twitter.com/"+indict['twitter_handle']+"/"+str(indict['id'])
                )
            
            db.session.add(a_tweet)


db.session.commit()
db.session.close()


org = Organization.query.first()
project = Project(name="DF og Ehl", organization=org.id)
db.session.add(project)
db.session.commit()

print(Tweet.query.all())

for t in Tweet.query.all():
    print(t)

/home/arthur/Documents/GitHub/nlp4all/nlp4all/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


[<Tweet 1>, <Tweet 2>, <Tweet 3>, <Tweet 4>, <Tweet 5>, <Tweet 6>, <Tweet 7>, <Tweet 8>, <Tweet 9>, <Tweet 10>, <Tweet 11>, <Tweet 12>, <Tweet 13>, <Tweet 14>, <Tweet 15>, <Tweet 16>, <Tweet 17>, <Tweet 18>, <Tweet 19>, <Tweet 20>, <Tweet 21>, <Tweet 22>, <Tweet 23>, <Tweet 24>, <Tweet 25>, <Tweet 26>, <Tweet 27>, <Tweet 28>, <Tweet 29>, <Tweet 30>, <Tweet 31>, <Tweet 32>, <Tweet 33>, <Tweet 34>, <Tweet 35>, <Tweet 36>, <Tweet 37>, <Tweet 38>, <Tweet 39>, <Tweet 40>, <Tweet 41>, <Tweet 42>, <Tweet 43>, <Tweet 44>, <Tweet 45>, <Tweet 46>, <Tweet 47>, <Tweet 48>, <Tweet 49>, <Tweet 50>, <Tweet 51>, <Tweet 52>, <Tweet 53>, <Tweet 54>, <Tweet 55>, <Tweet 56>, <Tweet 57>, <Tweet 58>, <Tweet 59>, <Tweet 60>, <Tweet 61>, <Tweet 62>, <Tweet 63>, <Tweet 64>, <Tweet 65>, <Tweet 66>, <Tweet 67>, <Tweet 68>, <Tweet 69>, <Tweet 70>, <Tweet 71>, <Tweet 72>, <Tweet 73>, <Tweet 74>, <Tweet 75>, <Tweet 76>, <Tweet 77>, <Tweet 78>, <Tweet 79>, <Tweet 80>, <Tweet 81>, <Tweet 82>, <Tweet 83>, <Tweet 84>, 

In [10]:
 d = datetime('')

TypeError: an integer is required (got type str)

In [11]:
dmindb.session.close()

In [12]:
dt = datetime.now()
tweet.date_posted = dt

db.session.add(tweet)

AttributeError: 'int' object has no attribute 'date_posted'

In [13]:
db.session.commit()

In [14]:
db.session.close()

In [15]:
db.drop_all()

In [16]:
TweetTagDescription.query.all()

OperationalError: (sqlite3.OperationalError) no such table: tweet_tag_description
[SQL: SELECT tweet_tag_description.id AS tweet_tag_description_id, tweet_tag_description.name AS tweet_tag_description_name, tweet_tag_description.text AS tweet_tag_description_text 
FROM tweet_tag_description]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [17]:
db.session.rollback()

In [22]:
ttd = TweetTagDescription(name="test1", text="lololol")
db.session.add(ttd)


In [23]:
db.session.commit()

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.OperationalError) no such table: tweet_tag_description
[SQL: INSERT INTO tweet_tag_description (name, text) VALUES (?, ?)]
[parameters: ('test1', 'lololol')]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [20]:
db.create_all()

<function sqlalchemy.expression.table(name, *columns)>

In [12]:
for tweet in Tweet.query.all():
    print(tweet.content)

something racist
Nedlukning af regionerne flytter 1,5 mia kr fra administration til sundhed. God liberal politik 👍🏻😀🇩🇰 #dkpol
Vi har nu landet en sundhedsreform, som sætter patienternes rettigheder i fokus, og som fjerner en stor klump unødig bøvl og bureaukrati.

Læs mere her 👉 https://t.co/YEtvWqo1eL #sundpol #dkpol https://t.co/3hh5TRn6gs
I et ny afsnit af vores podcast kan du høre, hvordan det gik, da @SociologenHD og William Kvist mødtes i en samtale, man kunne fristes til at give overskriften ’Kloge Ågerne’.

Hør podcasten her 👉 https://t.co/gdIwWMuEFq https://t.co/HGvHyHpH6B
“Udenrigsminister Anders Samuelsen viser en ny dansk stil i FN. Det er godt nyt for menneskerettighederne.” #dkpol #dkmedier 

https://t.co/xy6yNQElr1
Det er fundamentalt forkert, at Israel som det eneste land i verden har et helt dagsordenspunkt dedikeret til sig i FN’s Menneskerettighedsråd. Derfor vil Danmark i morgen - af princip - stemme NEJ til alle resolutioner under Rådets dagsordenspunkt 7. #dkpol
L